In [1]:
import requests
import lxml.html as lh
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# refer url
url = 'https://www.worldometers.info/coronavirus/'

# create handle
page = requests.get(url)

# store the contents
doc = lh.fromstring(page.content)

In [3]:
tr_elements = doc.xpath('//tr')

In [4]:
# check the length of first 12 rows
[len(T) for T in tr_elements[:12]]

[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]

In [5]:
len(tr_elements[1])

19

In [6]:
# Parse table header
tr_elements = doc.xpath('//tr')

col=[]
i=0

In [7]:
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%s %d' % (name, i))
    col.append((name,[]))

# 1
Country,Other 2
TotalCases 3
NewCases 4
TotalDeaths 5
NewDeaths 6
TotalRecovered 7
NewRecovered 8
ActiveCases 9
Serious,Critical 10
Tot Cases/1M pop 11
Deaths/1M pop 12
TotalTests 13
Tests/
1M pop
 14
Population 15
Continent 16
1 Caseevery X ppl 17
1 Deathevery X ppl 18
1 Testevery X ppl 19


In [8]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=len(tr_elements[1]):
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [9]:
[len(C) for (title,C) in col]

[695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695,
 695]

In [10]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

In [11]:
df = df[['Country,Other', 'TotalCases']]

In [12]:
df.head()

,"Country,Other",TotalCases
0,\nNorth America\n,"2,512,457"
1,\nSouth America\n,"1,463,160"
2,\nEurope\n,"2,229,528"
3,\nAsia\n,"1,667,018"
4,\nAfrica\n,"254,339"


In [13]:
map = gpd.read_file('World_Map.shp')

In [14]:
map.head()

,NAME,geometry
0,Antigua and Barbuda,"MULTIPOLYGON (((-61.68667 17.02444, -61.73806 ..."
1,Algeria,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ..."
2,Azerbaijan,"MULTIPOLYGON (((45.08332 39.76804, 45.26639 39..."
3,Albania,"POLYGON ((19.43621 41.02107, 19.45055 41.06000..."
4,Armenia,"MULTIPOLYGON (((45.57305 40.63249, 45.52888 40..."


In [15]:
df_notin_map = []
for i in df['Country,Other'].tolist():
    map_list = map['NAME'].tolist()
    if i in  map_list:
        pass
    else:
        df_notin_map.append(i)

In [16]:
df1 = df.set_index('Country,Other')

In [17]:
df1 = df1.drop(df_notin_map)

In [18]:
df1 = df1.sort_values(by='Country,Other')

In [19]:
df1.head()

,TotalCases
"Country,Other",
Afghanistan,"25,623"
Afghanistan,"24,766"
Afghanistan,"25,527"
Albania,"1,590"
Albania,"1,590"


In [20]:
df1.duplicated()

Country,Other
Afghanistan    False
Afghanistan    False
Afghanistan    False
Albania        False
Albania         True
               ...  
Zambia          True
Zambia         False
Zimbabwe       False
Zimbabwe       False
Zimbabwe        True
Length: 537, dtype: bool

In [21]:
df1.drop_duplicates(keep='last')

,TotalCases
"Country,Other",
Afghanistan,"25,623"
Afghanistan,"24,766"
Afghanistan,"25,527"
Albania,"1,590"
Albania,"1,521"
...,...
Yemen,728
Zambia,"1,382"
Zambia,"1,358"


In [22]:
df1 = df1[~df1.index.duplicated(keep='last')]

In [23]:
df1.head()

,TotalCases
"Country,Other",
Afghanistan,"25,527"
Albania,"1,521"
Algeria,"10,919"
Andorra,853
Angola,140


# Create Dataframe with these countries as index

 'USA',
 'UK',
 'Iran',
 'UAE',
 'S. Korea',
 'Moldova',
 'Czechia',
 'Ivory Coast',
 'DRC',
 'North Macedonia',
 'CAR',
 'South Sudan',
 'Cabo Verde',
 'Diamond Princess',
 'Channel Islands',
 'Tanzania',
 'Eswatini',
 'Réunion',
 'Libya',
 'Vietnam',
 'Myanmar',
 'Faeroe Islands',
 'Syria',
 'Brunei ',
 'Sint Maarten',
 'Macao',
 'St. Vincent Grenadines',
 'Curaçao',
 'Laos',
 'Falkland Islands',
 'Turks and Caicos',
 'Vatican City',
 'MS Zaandam',
 'Caribbean Netherlands',
 'St. Barth',
 'Saint Pierre Miquelon',


In [24]:
map.sort_values('NAME')
map.head(3)

,NAME,geometry
0,Antigua and Barbuda,"MULTIPOLYGON (((-61.68667 17.02444, -61.73806 ..."
1,Algeria,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ..."
2,Azerbaijan,"MULTIPOLYGON (((45.08332 39.76804, 45.26639 39..."


## These problems still exist
1. Banyak yang belum sinkron di `Country,Other`
2. Ada dua duplicates dalam setiap cacat di `Country,Other`

In [25]:
df2 = df.set_index('Country,Other')

In [26]:
df2 = df2.loc[['USA', 'UK', 'Iran', 'UAE', 'S. Korea', 'Moldova', 'Czechia', 'Ivory Coast', 'DRC', 'North Macedonia', 'CAR', 'South Sudan', 'Cabo Verde', 'Diamond Princess', 'Channel Islands', 'Tanzania', 'Eswatini', 'Réunion', 'Libya', 'Vietnam', 'Myanmar', 'Faeroe Islands', 'Syria', 'Brunei ', 'Sint Maarten', 'Macao', 'St. Vincent Grenadines', 'Curaçao', 'Laos', 'Falkland Islands', 'Turks and Caicos', 'Vatican City', 'MS Zaandam', 'Caribbean Netherlands', 'St. Barth', 'Saint Pierre Miquelon']]
df2.head()

,TotalCases
"Country,Other",
USA,"2,182,951"
USA,"2,182,950"
USA,"2,162,228"
UK,"296,857"
UK,"296,857"


In [27]:
df2.sort_values('Country,Other').head(9)

,TotalCases
"Country,Other",
Brunei,141
Brunei,141
Brunei,141
CAR,"2,289"
CAR,"2,222"
CAR,"2,057"
Cabo Verde,760
Cabo Verde,760
Cabo Verde,750


In [28]:
df2 = df2[~df2.index.duplicated(keep='first')]

In [29]:
df2.head()

,TotalCases
"Country,Other",
USA,"2,182,951"
UK,"296,857"
Iran,"189,876"
UAE,"42,636"
S. Korea,"12,155"


In [30]:
df2 = df2.rename(
    index={
        'USA': 'United States',
        'UK': 'United Kingdom',
        'Iran': 'Iran (Islamic Republic of)',
        'UAE': 'United Arab Emirates',
        'S. Korea': 'Korea, Republic of',
        'Moldova': 'Republic of Moldova',	
        'Ivory Coast': "Cote d'Ivoire",
        'DRC': 'Democratic Republic of the Congo',
        'North Macedonia': 'The former Yugoslav Republic of Macedonia',	
        'CAR': 'Central African Republic',
        'Cabo Verde': 'Cape Verde',		
        'Tanzania': 'United Republic of Tanzania',
        'Eswatini': 'Swaziland',
        'Libya': 'Libyan Arab Jamahiriya',
        'Vietnam':'Viet Nam',	
        'Myanmar':'Burma',	
        'Faeroe Islands':'Faroe Islands',	
        'Syria':'Syrian Arab Republic',	
        'Brunei':'Brunei Darussalam',	
        'Macao':'Macau',	
        'St. Vincent Grenadines':'Saint Vincent and the Grenadines',	
        'Laos':"Lao People's Democratic Republic",	
        'Falkland Islands':'Falkland Islands (Malvinas)',	
        'Turks and Caicos':'Turks and Caicos Islands',	
        'Vatican City':'Holy See (Vatican City)',		
        'St. Barth':'Saint Barthelemy',
        'Saint Pierre Miquelon':'Saint Pierre and Miquelon'
        })

In [31]:
# Find similar values
same_val = [col for col in map.NAME if 'Miquelon' in col]
same_val

['Saint Pierre and Miquelon']

In [32]:
df2.head()

,TotalCases
"Country,Other",
United States,"2,182,951"
United Kingdom,"296,857"
Iran (Islamic Republic of),"189,876"
United Arab Emirates,"42,636"
"Korea, Republic of","12,155"


In [33]:
df_merge = pd.concat([df2,df1])

In [34]:
df_merge.shape

(215, 1)

In [35]:
df_merge.nunique()

TotalCases    203
dtype: int64

In [39]:
df_merge.head(10)

,TotalCases
"Country,Other",
United States,"2,182,951"
United Kingdom,"296,857"
Iran (Islamic Republic of),"189,876"
United Arab Emirates,"42,636"
"Korea, Republic of","12,155"
Republic of Moldova,"11,879"
Czechia,"10,066"
Cote d'Ivoire,"5,439"
Democratic Republic of the Congo,"4,837"
